# Distributed IO with a larger cluster

We'll see that for applications that are limited by IO bandwidth, a wide distribution across compute nodes can be beneficial.

## Technical preamble 

Spin up cluster using `salloc` and `srun`as described here: <https://gist.github.com/willirath/772e0de2b6fbe845f77388c3b16390ea>

We'll use
```shell
salloc -N 50 -n 50 -c 24 -A training2005 --time=00:15:00
```
And therein (TODO: How to escape env vars in the `srun` calls?)
```shell
$ conda activate py3_dask
$ srun -r0 -n1 -N1 bash -c 'dask-scheduler --scheduler-file scheduler.json --host ${SLURMD_NODENAME}.ib.juwels.fzj.de &> scheduler.log' &
$ srun -n49 -N49 --cpus-per-task=96 bash -c 'dask-worker --scheduler-file scheduler.json --nthreads=16 --memory-limit=96GB --local-directory=/tmp/ --host ${SLURMD_NODENAME}.ib.juwels.fzj.de &>> worker.log' &
```

In [ ]:
import dask, dask.distributed, os

In [ ]:
client = dask.distributed.Client(scheduler_file="scheduler.json")

In [ ]:
client

## Create random data and write them to disk

In [ ]:
from dask import array as darr

In [ ]:
random_data = darr.random.normal(
    size=(int(2_000_000_000_000 / 8), ),
    chunks=(int(1_000_000_000 / 8), )
)
random_data

In [ ]:
!rm -rf random_data.zarr/

In [ ]:
%time random_data.to_zarr("random_data.zarr")

In [ ]:
!du -sh random_data.zarr/

## Find largest number with disk IO

We'll re-read the data and find the maximum on the fly.

Note in the Dask dashboard that we don't saturate CPU load.
This means we're limited by IO rather than compute.

In [ ]:
random_data = darr.from_zarr("random_data.zarr/")
random_data

In [ ]:
%time random_data.max().compute()

## Bottom line

For IO bound problems, we'd like to be able to scale horizontally rather than vertically.

That's something that could be tackled with the scheduler config (fill all nodes equally vs. keep as many nodes as possible empty).